In [ ]:
import random
random.seed(42)

In [ ]:
!mkdir spotGEO

In [ ]:
!mkdir spotGEO/test

In [ ]:
!mkdir spotGEO/test/images

In [ ]:
!mkdir spotGEO/test/detections

In [ ]:
!mkdir spotGEO/data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/spotGEO/test.zip' -d '/content/spotGEO/data/'

In [ ]:
from PIL import Image

In [ ]:
def copy_test_images(read_path, write_path, start_seq_id, end_seq_id):
  for seq_id in range(start_seq_id,end_seq_id+1):
    for frm_id in range(1,6):
        im = Image.open(read_path+str(seq_id)+'/'+str(frm_id)+'.png')
        rgb_im = im.convert('RGB')
        rgb_im.save(write_path+'seq_'+str(seq_id)+'_frm_'+str(frm_id)+'.jpg')

In [ ]:
%%time

copy_test_images('/content/spotGEO/data/test/', '/content/spotGEO/test/images/', 2561, 5120)

In [ ]:
!pip3 install tensorflow-gpu==1.13.1

In [ ]:
!pip3 install keras==2.2.4

In [ ]:
!pip3 install numpy==1.16.1

In [ ]:
!pip3 install imageai --upgrade

In [ ]:
%tensorflow_version 1.x

In [ ]:
%%time

from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath('/content/drive/My Drive/spotGEO/detection_model-ex-019--loss-0015.354.h5') 
detector.setJsonPath('/content/drive/My Drive/spotGEO/detection_config.json')
detector.loadModel()

In [ ]:
%%time

detections = detector.detectObjectsFromImage(input_image='spotGEO/test/images/seq_2561_frm_1.jpg', output_image_path='spotGEO/test/detections/test.jpg', minimum_percentage_probability=10)
for detection in detections:
    print(detection['name'], ' : ', detection['percentage_probability'], ' : ', detection['box_points'])

In [ ]:
len(detections)

In [ ]:
detections[0]

In [ ]:
import json

In [ ]:
%%time

submission = []

for seq_id in range(2561,5121):
  for frm_id in range(1,6):

    detections = detector.detectObjectsFromImage(input_image='spotGEO/test/images/seq_'+str(seq_id)+'_frm_'+str(frm_id)+'.jpg', output_image_path='spotGEO/test/detections/test.jpg', minimum_percentage_probability=10)

    detections = sorted(detections, key=lambda k: k['percentage_probability'], reverse=True)[:30]

    object_coords = []

    for i in range(0,len(detections)):

      coords = []

      xmin = detections[i]['box_points'][0]
      xmax = detections[i]['box_points'][2]
      ymin = detections[i]['box_points'][1]
      ymax = detections[i]['box_points'][3]

      x = (xmin + xmax) / 2
      y = (ymin + ymax) / 2

      coords.extend([x, y])

      object_coords.append(coords)

    submission.append({'sequence_id' : seq_id,
                       'frame' : frm_id,
                       'num_objects' : len(object_coords),
                       'object_coords' : object_coords})

with open('spotGEO/test/detections/my_submission_2561_5120.json', 'w') as outfile:
    json.dump(submission, outfile)

In [ ]:
!cp 'spotGEO/test/detections/my_submission_2561_5120.json' -d '/content/drive/My Drive/spotGEO/'